# Cycle 9: Testing and Test-Driven Development

In this cycle we'll be talking about approaches to testing your code, and a development style inspired by testing.  

We'll be using `assert`s as we learned about in the exceptions and error handling section to build our tests, and talk about regression testing.  

After this cycle, I would like you to be able to:
1.   Devise a set of tests for a piece of code and describe why you've chosen those tests
2.   Use `assert`s to set up testing code
3.   Say what test-driven development is and why some people think it is a good approach.
4.  Use test-driven development to devise tests and then code to solve a specified problem.  

Some material here is adapted from or inspired by: https://swcarpentry.github.io/python-novice-inflammation/10-defensive/index.html


## Let's talk testing

We often manually test our code as we're working, and this is entirely normal.  We can also be a bit more systematic and automated about this, and this can help us when we're writing larger pieces of code that may change over time.  

When trying to write a suite of tests for code, it's good to try to include tests with:
- typical values
- 'edge' or 'corner' cases - cases at the boundary of normal, or odd in some way (e.g. the number `0` for a function that operates on all non-negative integers)
- exceptional cases (e.g. negative numbers for a function that operates on positive integers, malformed input, etc)

Let's look at a very simple example:


In [16]:
# takes a list as an argument and returns 
# the largest value in the list
# or returns None if the list is empty or there's an exception
def find_maximum_value(my_list):
  try:
    if len(my_list) == 0:
      return None
    max_so_far = my_list[0]
    for item in my_list:
      if max_so_far < item:
        max_so_far = item
    return max_so_far
  except Exception as e:
    print('WARNING:Exception thrown ' + str(e))
    return None

What sorts of test inputs might we want to include?
- typical inputs of various types:
  - list with largest value at beginning, end, in the middle
  - list with repeated values
- corner cases, like an empty list
- exceptional input: e.g. function as input

What about something more exotic, like a list with mixed types of things?  Here I'll call this an exceptional case, but of course it depends on the design and the code! What about strings or dictionaries as input?  Not really well specified here.  

Let's write some tests:

In [13]:
print(find_maximum_value([]))
print(find_maximum_value([1, 2, 3]))
print(find_maximum_value([9, 1, 2, 3]))
print(find_maximum_value(['a', 'b', 's', 'a', 'd']))
print(find_maximum_value(print))

None
3
9
s
Exception thrown object of type 'builtin_function_or_method' has no len()
None


While the coverage is pretty good, the trouble here is that we need to visually inspect the printed output.  If we were dealing with more code that changed over time, thsi could be cumbersome if we want to re-run the tests often.  

We can automate this in many ways - we'll use asserts.  

In [15]:
def test_maximum_value_typical():
  assert find_maximum_value([1, 2, 3]) == 3
  assert find_maximum_value([9, 1, 2, 3]) == 9
  assert find_maximum_value(['a', 'b', 's', 'a', 'd']) == 's'

def test_maximum_value_empty():
  assert find_maximum_value([]) == None

def test_maximum_value_bad_input():
  assert find_maximum_value(print) == None

test_maximum_value_typical()
test_maximum_value_empty()
test_maximum_value_bad_input()

9
Exception thrown object of type 'builtin_function_or_method' has no len()


This silently works if all is well, and fails loudly if something goes wrong.  If we wanted to, we could add print statements to try/except blocks to be more reassuring.

In [17]:
def test_maximum_value_typical():
  assert find_maximum_value([1, 2, 3]) == 3
  assert find_maximum_value([9, 1, 2, 3]) == 9
  assert find_maximum_value(['a', 'b', 's', 'a', 'd']) == 's'
  print('test_maximum_value_typical successful')
  # or we could have a success/fail return value

test_maximum_value_typical()

 test_maximum_value_typical successful


This sort of approach can hook into testing programs (e.g. pytest).  We won't be using pytest in 1PX ***and I won't test you on it***, but in the video just for fun I'll give you a peek at how we use pytest with a testing setup in one of my projects.

## Test-driven development

Since we've thought so much about tests let's talk about **test-driven development**.

The idea of test-driven development is that we write the tests first and the code after. People who like this approach think that this ordering helps programmers figure out what a piece of code is supposed to do before starting to code, and avoids confirmation bias in test writing. 

Let's work on an example from the software carpentry site: a function called `range_overlap` that tells us whether two or more ranges overlap:
python-overlapping-ranges.svg

(image from https://swcarpentry.github.io/python-novice-inflammation/10-defensive/index.html)

Let's write some test cases first:


In [27]:
def test_range_overlap_no_intersect():
    assert range_overlap([ (0.0, 1.0), (5.0, 6.0) ]) == None

def test_range_overlap_no_intersect_closed():
    assert range_overlap([ (0.0, 1.0), (1.0, 2.0) ]) == None

def test_range_overlap_single_range():
    assert range_overlap([ (0.0, 1.0) ]) == (0.0, 1.0)

def test_range_overlap_subset():
    assert range_overlap([ (2.0, 3.0), (2.0, 4.0) ]) == (2.0, 3.0)

def test_range_overlap_three_with_neg():
    assert range_overlap([ (0.0, 1.0), (0.0, 2.0), (-1.0, 1.0) ]) == (0.0, 1.0)

def test_range_overlap_no_ranges():
    assert range_overlap([]) == None


test_range_overlap_single_range()
test_range_overlap_subset()
test_range_overlap_three_with_neg()
test_range_overlap_no_ranges()
test_range_overlap_no_intersect()
test_range_overlap_no_intersect_closed()

Of course we can't run this yet, because we haven't implemented `range_overlap`

Looking back at our picture for inspiration, we can now write our code:
python-overlapping-ranges.svg

(image/code adapted from Software Carpentry)

In [25]:
def range_overlap(ranges):
    if len(ranges) == 0:
      return None
    (max_left, min_right) = ranges[0]
    for (left, right) in ranges:
        max_left = max(max_left, left)
        min_right = min(min_right, right)
    if max_left >= min_right:
      return None
    else:
       return (max_left, min_right)
